In [1]:
def find_spans (text, layer_name):
    layer = getattr(text, layer_name)
    spancount = len(layer.spans)
    spanindexes = []
    text=text.text
    attributes = layer.attributes    
    
    for s in layer.spans:
        spanstart = s.start
        spanend = s.end
        spanindexes.append(spanstart)
        spanindexes.append(spanend)
    
    html_spans=[]
    
    spanindexes = sorted(set(spanindexes))
    
    for i in range(len(spanindexes)-1):
        html_spans.append([spanindexes[i],spanindexes[i+1]])
            
    printable=[]
    
    for i, html_span in enumerate(html_spans):
        printable.append([])
        for s in layer.spans:
            if html_span[0] in range(s.start,s.end):
                for attribute in attributes:
                    printable[i].append(getattr(s,attribute))

    assert len(html_spans)==len(printable)
    
    for i, attributes in enumerate(printable):
        if len(attributes)==0:
            printable.remove(attributes)
            html_spans.remove(html_spans[i])

    return html_spans,printable

In [65]:
def add_spans(text,layer):
    t = text.text
    spans = find_spans(text,layer)[0]

    contents = find_spans(text,layer)[1]
    attributes = getattr(text,layer).attributes
    fragments = []
    last_end = 0
    new_contents = []
    
    for content in contents:
        new_content=[]
        for i in range(len(attributes)):
            new_content.append(''.join([attributes[i],": ",str(content[i])]))
        new_contents.append(new_content)
        
    contents = new_contents
    
    for i, (s, e) in enumerate(spans):
        fragments.append(t[last_end:s])
        fragments.append('<span id="test" onclick="box{0}(')
        fragments.append('`')
        fragments.append(str(contents[i]).replace('"',''))
        fragments.append('`,')
        fragments.append(str(i))
        fragments.append(')">')
        fragments.append(t[s:e])
        fragments.append('</span>')
        last_end = e
    fragments.append(t[last_end:])
    return ''.join(fragments)

In [61]:
t = '''
<html>
<style>
.boxes {{
    background: linear-gradient(to bottom right, white, lightgray);
    border: 1px solid black;
    border-radius: 4px;
    box-shadow: 4px 4px 4px gray;
    width: fit-content;
    position: relative;
}}

</style>
<body>

<p id="maintext{0}">{1}</p>

<script>

function box{0}(content,index) {{
    let textposition = maintext{0}.getBoundingClientRect().left;
    let boxposition = test[index].getBoundingClientRect().left-textposition;
    let textbox = document.createElement("DIV");
    textbox.setAttribute("id", "boxtest");
    textbox.classList.add("boxes");
    let t = document.createTextNode(content);
    textbox.appendChild(t);
    let textNode{0} = document.getElementById("maintext{0}");
    let previousBox = document.querySelector('.boxes');
    if (previousBox){{
        previousBox.parentNode.removeChild(previousBox);
    }}
    textNode{0}.appendChild(textbox);
    console.log(boxposition);
    document.getElementById("boxtest").style.left=boxposition+"px";
}}

</script>

</body>
</html>
'''


In [68]:
from IPython.display import display_html
from estnltk import Layer

class EstnltkDisplay:
    """Fancy display for estnltk objects."""

    _text_id = 0

    def layer_to_html(self, text, layer, text_id):
        result = add_spans(text, layer.name)
        result = result.format(text_id)
        return t.format(text_id,result)

    def display_layer(self, text, layer):
        self.__class__._text_id += 1
        html = self.layer_to_html(text, layer, self.__class__._text_id)
        display_html(html, raw=True)

    def __call__(self, obj):
        if isinstance(obj, Layer):
            self.display_layer(obj.text_object, obj)
        else:
            raise NotImplementedError('bla bla')

display = EstnltkDisplay()


from estnltk.tests import new_text

display(new_text(5).layer_1)

Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd tuhat.

In [67]:
display(new_text(5).layer_1)

<span id="test" onclick="box2(`[attr: ['L1-0'], attr_1: ['SADA']]`,0)">Sada</span> <span id="test" onclick="box2(`[attr: ['L1-1'], attr_1: ['KAKS']]`,1)">kaks</span><span id="test" onclick="box2(`[attr: ['L1-2', 'L1-2', 'L1-2'], attr_1: ['KAKS', 'KÜMME', 'KAKSKÜMMEND']]`,2)">kümme</span><span id="test" onclick="box2(`[attr: ['L1-2', 'L1-2', 'L1-2'], attr_1: ['KAKS', 'KÜMME', 'KAKSKÜMMEND']]`,3)">nd</span> <span id="test" onclick="box2(`[attr: ['L1-4'], attr_1: ['KOLM']]`,4)">kolm</span>.<span id="test" onclick="box2(`[attr: ['L1-5'], attr_1: ['NELI']]`,5)"> Neli</span> <span id="test" onclick="box2(`[attr: ['L1-6'], attr_1: ['TUHAT']]`,6)">tuhat</span> <span id="test" onclick="box2(`[attr: ['L1-7'], attr_1: ['VIIS']]`,7)">viis</span><span id="test" onclick="box2(`[attr: ['L1-8', 'L1-8', 'L1-8'], attr_1: ['SADA', 'VIIS', 'VIISSADA']]`,8)">sada</span> <span id="test" onclick="box2(`[attr: ['L1-10'], attr_1: ['KUUS']]`,9)">kuus</span><span id="test" onclick="box2(`[attr: ['L1-11', 'L1-11'

Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd tuhat.